This runs the MLClassifer on guanyi's full followers dataset. This shouldn't be used because the huggingface example code is better

In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd "/content/drive/MyDrive/WPI/Junior/IQP/Machine Models/data/v4"

/content/drive/MyDrive/WPI/Junior/IQP/Machine Models/data/v4


In [3]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 15.2 MB/s 
     |████████████████████████████████| 6.8 MB 34.7 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 45.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
# Get device to use
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
import torch_xla
import torch_xla.core.xla_model as xm
device = xm.xla_device()

     |████████████████████████████████| 149.9 MB 28 kB/s 
     |████████████████████████████████| 831.4 MB 2.1 kB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.10
    Uninstalling google-api-python-client-1.12.10:
      Successfully uninstalled google-api-python-client-1.12.10
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 

In [8]:
import json
import torch
import numpy as np
import tensorflow as tf
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [9]:
# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

# tpu_worker = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466') 
# print(tf.profiler.experimental.client.monitor(tpu_worker,1))

In [10]:
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
!ls results/checkpoint-12000

ls: cannot access 'results/checkpoint-12000': No such file or directory


In [12]:
model_path = "results/checkpoint-450"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2).to(device)

In [13]:
training_args = TrainingArguments(per_device_eval_batch_size=1024, per_device_train_batch_size=1024, output_dir=".")
test_trainer = Trainer(model, args=training_args)

In [14]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [15]:
#remove punctuation
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

table = str.maketrans('', '', string.punctuation)

# Remove stop words
from nltk.corpus import stopwords

stop = stopwords.words('english')

# Lemmatize all words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [16]:
# followers_timeline_file = open("../followers_timeline_full/userTimeline_{}.json".format(str(0)), 'r')
# followers_timeline = json.load(followers_timeline_file)

In [17]:
# for user in followers_timeline:
#   print(str(user))
#   break



In [18]:
# user_data = followers_timeline["1004425142"][2]["data"][0]["text"]
# print(user_data)

In [22]:
maybe_bad_tweets = {}
maybe_bad_tweets["tweets"] = []
tweet_count = 0
for i in range(1000):
 

  try:
    followers_timeline_file = open("../followers_timeline_full/userTimeline_{}.json".format(str(i)), 'r')
    print("Working on file {}".format(str(i)))
  except:
    print("File not found")
    break
  followers_timeline = json.load(followers_timeline_file)
  print(len(followers_timeline))
  tweet_text_list = []
  tweets_list = []
  for user in followers_timeline:
    # print(str(user))
    for user_slice in user:
      # print(str(user_slice))
      # print(followers_timeline[user][int(user_slice)])
      try:
      # print(len(followers_timeline[user][user_slice]["data"]))
        for tweet in followers_timeline[user][int(user_slice)]["data"]:
          tweet_text = tweet['text']
          tweet_text = ' '.join([word for word in tweet_text.lower().split() if word not in stop])  # remove stopwords
          # remove punc.
          tweet_text = tweet_text.translate(table)
          tweet_text = ''.join(c for c in tweet_text if not c.isdigit())  # remove numbers
          tweet_text = tweet_text.replace('  ', ' ')  # remove double spaces
          tweet_text = ' '.join([lemmatizer.lemmatize(word) for word in tweet_text.split()])  # lemminize
          tweet_text_list.append(tweet_text)
          tweets_list.append(tweet)
          tweet_count += 1
      except:
        # pass
        print(str(user))
    # print(followers_timeline[user]['data'])
    
    # except:
    #     pass

  # print(len(tweet_text_list))

  # X_test_tokenized = tokenizer(tweet_text_list, padding=True, truncation=True, max_length=512)
  # test_dataset = Dataset(X_test_tokenized)
  # raw_pred, tmp1, tmp2 = test_trainer.predict(test_dataset)

  # y_prob = tf.math.softmax(raw_pred, axis=-1).numpy()
  # y_pred = np.argmax(raw_pred, axis=1)
  # counter = 0
  # for tweet_pred in y_prob:
  #     # print(tweet_pred[1])
  #     if tweet_pred[1] >= np.float32(0.95):
  #         print(tweet_text_list[counter])
  #         print(type(tweet_pred[1]))
  #         print(tweet_pred[1])
  #         tweets_list[counter]["confidence"] = tweet_pred[1].item()
  #         maybe_bad_tweets["tweets"].append(tweets_list[counter])
  #     counter += 1
  # # Sort by confidence
  # maybe_bad_tweets["tweets"] = sorted(maybe_bad_tweets["tweets"], key=lambda k: k['confidence'], reverse=True)

  # # Write maybe bad tweets to file
  # with open("maybe_bad_tweets_v8_{}.json".format(str(i)), 'w') as outfile:
  #   json.dump(maybe_bad_tweets, outfile)
  
print(str(tweet_count))

Working on file 0
100
1007067202644611073
1007067202644611073
1007067202644611073
1007067202644611073
1007067202644611073
1007067202644611073
1007067202644611073
1007067202644611073
1007067202644611073
1288932932
1288932932
1288932932
1288932932
1288932932
1288932932
1408510868
1408510868
1408510868
1408510868
1408510868
1408510868
1408510868
1408510868
1716742992
1716742992
1716742992
1716742992
1716742992
1920965323
1920965323
1920965323
1920965323
228644510
228644510
228644510
228644510
228644510
231919427
231919427
231919427
231919427
231919427
231919427
231919427
231919427
231919427
2338199540
2338199540
2338199540
2365959104
2365959104
2365959104
2368728967
2368728967
2368728967
2368728967
2368728967
2368728967
2368728967
2368728967
2368728967
2368728967
2387781259
2387781259
2387781259
2387781259
2387781259
2632487201
2632487201
2632487201
2632487201
2632487201
2632487201
2632487201
2632487201
2632487201
2632487201
2657109938
2657109938
2657109938
2657109938
2657109938
270018626

In [23]:
# Write maybe bad tweets to file
print(str(tweet_count))

1236361
